In [45]:
with open("d6_input.txt", "r", encoding="utf8") as f:
    text = [x.strip("\n") for x in f.readlines()]
text[:10]

['.........................#...........#.#...#...............#..............##.....................#...........#..#.........#.......',
 '.....................#.................#.......#.........................#...........#.#.......#..................................',
 '.........#...........#........#.........#.........#...........#........................................#.........................#',
 '.......#.........................#...........................#.............#......................................................',
 '..................#................##...............#.#.....................##............#.......................................',
 '......#...#................#..................................................................#....#.#............................',
 '.......##................#........#..#.......#......................................#........#............#......###..........#...',
 '.....................#...............................

In [46]:
import itertools
hashes = list(itertools.chain.from_iterable([[(row, i) for i, e in enumerate(row_text) if e == "#"] for row, row_text in enumerate(text)]))
hashes[:10]

[(0, 25),
 (0, 37),
 (0, 39),
 (0, 43),
 (0, 59),
 (0, 74),
 (0, 75),
 (0, 97),
 (0, 109),
 (0, 112)]

In [47]:
by_row, by_col = [list() for _ in range(len(text))], [list() for _ in range(len(text[0]))]
for r, c in hashes:
    by_row[r].append(c)
    by_col[c].append(r)
by_row[:10]

[[25, 37, 39, 43, 59, 74, 75, 97, 109, 112, 122],
 [21, 39, 47, 73, 85, 87, 95],
 [9, 21, 30, 40, 50, 62, 103, 129],
 [7, 33, 61, 75],
 [18, 35, 36, 52, 54, 76, 77, 90],
 [6, 10, 27, 94, 99, 101],
 [7, 8, 25, 34, 37, 45, 84, 93, 106, 113, 114, 115, 126],
 [21, 74, 86, 89, 93, 120],
 [11, 16, 44, 46, 62, 65, 102, 117, 124],
 [5, 14, 15, 31, 39, 42, 89, 116]]

In [48]:
start = list(itertools.chain.from_iterable([[(row, i) for i, e in enumerate(row_text) if e in "^"] for row, row_text in enumerate(text)]))[0]
start

(94, 73)

In [49]:
from typing import Callable
dirs = {
    0: "up",
    1: "right",
    2: "down",
    3: "left"
}
next_dir: Callable[[int], int] = lambda x: (x+1)%4
next_dir(3)

0

In [50]:
import bisect
import collections
cols_in_row = collections.defaultdict(set)
walked, prev = 0, 0
facing = 0
rpos, cpos = start
while True:
    # print(f"{rpos=}, {cpos=}, {facing=}, {walked=}, new_walk={walked-prev}")
    prev = walked
    if facing == 0: # up, move within my col to a lower row
        x = bisect.bisect_left(by_col[cpos], rpos)
        if x == 0:
            # walked += rpos # if on row index 2, i go 3 spaces to fall off the edge
            for i in range(0, rpos+1):
                cols_in_row[i].add(cpos)
            break
        # walked += rpos - by_col[cpos][x-1] - 1
        for i in range(by_col[cpos][x-1]+1, rpos+1):
            cols_in_row[i].add(cpos)
        rpos = by_col[cpos][x-1] + 1
        
    elif facing == 1: # right, move within my row to a higher col
        x = bisect.bisect_right(by_row[rpos], cpos)
        if x == len(by_row[rpos]):
            for i in range(cpos, len(text[0])):
                cols_in_row[rpos].add(i)
            # walked += len(text[0]) - 1 - cpos # how many cols in a row minus my column position in my row (minus one)
            break
        # walked += by_row[rpos][x] - 1 - cpos
        for i in range(cpos, by_row[rpos][x]):
            cols_in_row[rpos].add(i)
        cpos = by_row[rpos][x] - 1

    elif facing == 2: # down, move within my col to a higher row
        x = bisect.bisect_right(by_col[cpos], rpos)
        if x == len(by_col[cpos]):
            # walked += len(text) - 1 - rpos
            for i in range(rpos, len(text)):
                cols_in_row[i].add(cpos)
            break
        # walked += by_col[cpos][x] - 1 - rpos
        for i in range(rpos, by_col[cpos][x]):
            cols_in_row[i].add(cpos)
        rpos = by_col[cpos][x] - 1

    elif facing == 3: # left, move within my row to a lower col
        x = bisect.bisect_left(by_row[rpos], cpos)
        if x == 0:
            # walked += cpos
            for i in range(0, cpos+1):
                cols_in_row[rpos].add(i)
            break
        # walked += cpos - by_row[rpos][x-1] - 1
        for i in range(by_row[rpos][x-1]+1, cpos+1):
            cols_in_row[rpos].add(i)
        cpos = by_row[rpos][x-1] + 1

    else:
        print("FAIL")
    facing = next_dir(facing)

# print(walked)
sum(len(x) for x in cols_in_row.values())

5242

a box is 
```
0x00x
00000
0x00x

(0, 1), (0, 4), (2, 1), (2, 4)


```

In [61]:
import itertools
colinear_pairs = [list(itertools.combinations(x, 2)) for x in by_row] 
colinear_pairs[0]


[(25, 37),
 (25, 39),
 (25, 43),
 (25, 59),
 (25, 74),
 (25, 75),
 (25, 97),
 (25, 109),
 (25, 112),
 (25, 122),
 (37, 39),
 (37, 43),
 (37, 59),
 (37, 74),
 (37, 75),
 (37, 97),
 (37, 109),
 (37, 112),
 (37, 122),
 (39, 43),
 (39, 59),
 (39, 74),
 (39, 75),
 (39, 97),
 (39, 109),
 (39, 112),
 (39, 122),
 (43, 59),
 (43, 74),
 (43, 75),
 (43, 97),
 (43, 109),
 (43, 112),
 (43, 122),
 (59, 74),
 (59, 75),
 (59, 97),
 (59, 109),
 (59, 112),
 (59, 122),
 (74, 75),
 (74, 97),
 (74, 109),
 (74, 112),
 (74, 122),
 (75, 97),
 (75, 109),
 (75, 112),
 (75, 122),
 (97, 109),
 (97, 112),
 (97, 122),
 (109, 112),
 (109, 122),
 (112, 122)]

In [60]:
# for each colinear pair, if we have another row with either point, that's a rectangle
rects = 0
for row_idx, colinear_row in enumerate(colinear_pairs):
    for a, b in colinear_row:
        if any([(a in x) or (b in x) for idx, x in enumerate(by_row) if idx != row_idx]):
            rects += 1
rects

2617

all that was not correct. i counted the number of rectangles we could create
but we are counting the loops, starting from the guard position that we can get stuck in
including non rectangles...


In [ ]:
by_row, by_col = [list() for _ in range(len(text))], [list() for _ in range(len(text[0]))]
for r, c in hashes:
    by_row[r].append(c)
    by_col[c].append(r)